In [2]:
import numpy as np
import pandas as pd
import os, sys

from sklearn.preprocessing import OneHotEncoder


In [3]:
pd.options.display.max_rows = 100

In [4]:
pd.reset_option('all')

In [5]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  preprocessedDataPath = '/content/drive/Othercomputers/Компьютер/predicting-academic-performance 2.0/v2.0/preprocessed_data'
  dataPath = '/content/drive/Othercomputers/Компьютер/predicting-academic-performance 2.0/data'
else:
  preprocessedDataPath = 'preprocessed_data'
  dataPath = '../data'

Смена энкодинга и сепаратора csv файлов

In [6]:
[fileName for fileName in os.listdir(path=dataPath) if os.path.splitext(fileName)[1]=='.csv']

['disciplina_chasy.csv',
 'discipliny.csv',
 'ist_status_student.csv',
 'ist_tip_fin_studenta.csv',
 'lichnost.csv',
 'propuski.csv',
 'status_group.csv',
 'students.csv',
 'students_tek.csv',
 'student_record_book.csv',
 'vid_zanyatiy.csv']

In [7]:
# fileName = 'discipliny.csv'
# df = pd.read_csv(os.path.join(dataPath, fileName), sep="#", encoding='cp1251')
# df.to_csv(os.path.join(dataPath, fileName), index=False)

Выкидывание всех не бакалавров очников и начавших обучение до 2010 года

In [8]:
st = pd.read_csv(os.path.join(dataPath, 'students.csv'))
st['Дата начала обучения'] = pd.to_datetime(st['Дата начала обучения'])
st = st[st['Дата начала обучения'] > pd.to_datetime('2010.1.1')]
ochno_bachelor_id = st[(st['Форма обучения'] == 'Очная') & \
              (st['Уровень образования'] == 'Бакалавр')]['LICHNOST_GUID'].unique()
st = st[st['LICHNOST_GUID'].isin(ochno_bachelor_id)]


Слуйчайный балл нормального распределения среди пропусков

In [9]:
def find_std_min_min_max_by_simvol(srb):
    mean_std_min_max_by_simvol = {}
    for key in srb['SIMVOL'].dropna().unique():
        mean_std_min_max_by_simvol[key] = (dict(srb.groupby('SIMVOL')['BALL'].mean())[key], \
                                            dict(srb.groupby('SIMVOL')['BALL'].std())[key], \
                                            dict(srb.groupby('SIMVOL')['BALL'].min())[key], \
                                            dict(srb.groupby('SIMVOL')['BALL'].max())[key])
    return mean_std_min_max_by_simvol

def simvol_to_ball(simvol, god_nabora, mean_std_min_max_by_simvol):
    mean, std, min, max = mean_std_min_max_by_simvol[int(god_nabora<=2017)][simvol]

    ball = round(np.clip(np.random.normal(mean, std), min, max))

    return ball

srb = pd.read_csv(os.path.join(dataPath, 'student_record_book.csv'))
srb_filtered_grades = srb[((((srb['BALL'].isna() == False) & (srb['BALL'] <= 100)) | (srb['OCENKA'].isna() == False)) & \
                           (srb['GOD_NABORA'].isna() == False) & \
                            (srb['LICHNOST_GUID'].isin(ochno_bachelor_id)))]

srb_filtered_grades['DATA_SDACHI'] = pd.to_datetime(srb_filtered_grades['DATA_SDACHI'], dayfirst=True, errors='coerce')


mean_std_min_max_by_simvol = [find_std_min_min_max_by_simvol(srb_filtered_grades[srb_filtered_grades['GOD_NABORA'] <= 2017]), \
                              find_std_min_min_max_by_simvol(srb_filtered_grades[srb_filtered_grades['GOD_NABORA'] > 2017])]

srb_fill_nan_ball = srb_filtered_grades.copy()
srb_fill_nan_ball['BALL'] = srb_fill_nan_ball.apply(lambda row: \
                                                    simvol_to_ball(row['SIMVOL'], row['GOD_NABORA'], mean_std_min_max_by_simvol) if pd.isnull(row['BALL']) else row['BALL'], axis=1)

C:\Users\fluke\AppData\Local\Temp\ipykernel_20452\922407441.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  srb_filtered_grades['DATA_SDACHI'] = pd.to_datetime(srb_filtered_grades['DATA_SDACHI'], dayfirst=True, errors='coerce')


Выкинуть записи пришедших после академа

In [10]:
iss = pd.read_csv(os.path.join(dataPath, 'ist_status_student.csv'))
iss['DATA_NACHALA'] = pd.to_datetime(iss['DATA_NACHALA'], errors='coerce')
iss_bach = iss[iss['LICHNOST_GUID'].isin(ochno_bachelor_id)]

In [11]:
iss_bach

,GRUPPA_GUID,LICHNOST_GUID,DATA_NACHALA,STATUS_STUDENTA_ID,STATUS,TIP_OSNOVANIYA_VVODA_ID,PRICHINA_VVODA_ZAPISI,PRICHINA_OTCHISLENIYA_ID,PRICHINA_OTCHISLENIYA,VID_OTPUSKA_ID,VID_OTPUSKA,TIP_PARAGRAFA_PRIKAZA_ID,OSNOVANIE_VVODA,KATEGORIYA_OBUCHAEMOGO_ID,KATEGORIYA_OBUCHAEMOGO
36490,b435984d-ea21-49e1-b852-c4d7f0069583,36233cc2-56a6-4750-a7a5-1d436a5a2cf3,2016-08-28,15,отчисленный,0,пр,13.0,окончание обучения,NaN,NaN,18.0,Присвоить,NaN,NaN
36491,b435984d-ea21-49e1-b852-c4d7f0069583,36233cc2-56a6-4750-a7a5-1d436a5a2cf3,2016-07-01,16,в отпуске,0,пр,NaN,NaN,11.0,каникулы,18.0,Присвоить,NaN,NaN
36492,b435984d-ea21-49e1-b852-c4d7f0069583,36233cc2-56a6-4750-a7a5-1d436a5a2cf3,2012-08-27,10,учащийся,0,пр,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN
37277,ce1721e1-387b-46db-8a3b-630820f5b2e6,0ac9834a-735c-46a2-a463-8f1eb630ea67,2020-09-01,10,учащийся,2,приказ,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN
37300,8c237755-c163-4b49-b07b-596f3e376b85,793930dc-22b1-4d00-ae5c-cc68ebc69e0a,2022-09-01,10,учащийся,2,приказ,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321191,ce6a470d-31ec-4bb8-a007-abd2871e926c,ed41df6a-5db2-409a-8bcd-74057efe588b,2023-09-01,10,учащийся,2,приказ,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN
321194,3ac7b23f-26f1-4f75-b3a0-268b353fd602,10438414-e973-4764-bc7d-7f40b45868cb,2023-10-13,15,отчисленный,0,пр,8.0,по собственному желанию,NaN,NaN,7.0,Отчислить,NaN,NaN
321195,3ac7b23f-26f1-4f75-b3a0-268b353fd602,10438414-e973-4764-bc7d-7f40b45868cb,2023-09-01,10,учащийся,2,приказ,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN
321197,51c1f800-954f-4a74-ab44-ce1dc94c98e3,eadf3acc-05db-47bd-9bcd-9d1c82edaec3,2023-09-01,10,учащийся,2,приказ,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN


Если встре

Находим id и date всех отчисленных по неуспеваемости

In [12]:
otchislennie_id_data = dict(iss_bach[~iss_bach['PRICHINA_OTCHISLENIYA'].isin(['болезнь',\
                                                          'нарушение правил прож. в общежитии',\
                                                            'нарушение правил внутр. распорядка',\
                                                                'окончание обуч. по сет. прог. с вузом РФ',\
                                                                    'завершен. программы обуч. академ. обмена'\
                                                                        'призыв в ряды РА',\
                                                                            'перевод в др. ВУЗ', 'окончание обучения'])].dropna(subset='PRICHINA_OTCHISLENIYA')[['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())

Находим id и date всех ушедших в академ

In [13]:
akademshiki_id_data = dict(iss_bach[iss_bach['VID_OTPUSKA'].isin(['академический (по семейным обстоятельствам)', 'академический (по болезни)', 'академический (по производственной необходимости)',\
                                                   'академический отпуск (по иным обстоятельствам)'])][['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())

Находим id и date всех тех, кто окончил обучение

In [14]:
zakonchili_obuchenie_id_data = dict(iss_bach[(iss_bach['PRICHINA_OTCHISLENIYA'] == 'окончание обучения') & (iss_bach['OSNOVANIE_VVODA'] == 'Присвоить')  ][['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())

Помечаем и удаляем все записи после окончания обучения, ухода в академ или отчисления

In [15]:
iss_bach['STATUS_STUDENTA_ID'] = iss_bach.apply(lambda row: -1 if (row['LICHNOST_GUID'] in akademshiki_id_data and akademshiki_id_data[row['LICHNOST_GUID']] < row['DATA_NACHALA']) or \
                                                         (row['LICHNOST_GUID'] in otchislennie_id_data and otchislennie_id_data[row['LICHNOST_GUID']] < row['DATA_NACHALA']) or \
                                                         (row['LICHNOST_GUID'] in zakonchili_obuchenie_id_data and zakonchili_obuchenie_id_data[row['LICHNOST_GUID']] < row['DATA_NACHALA'])
                                                           else row['STATUS_STUDENTA_ID'], axis=1)

iss_bach = iss_bach[iss_bach['STATUS_STUDENTA_ID'] != -1]


C:\Users\fluke\AppData\Local\Temp\ipykernel_20452\4215636021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iss_bach['STATUS_STUDENTA_ID'] = iss_bach.apply(lambda row: -1 if (row['LICHNOST_GUID'] in akademshiki_id_data and akademshiki_id_data[row['LICHNOST_GUID']] < row['DATA_NACHALA']) or \


Находим id и date заново

In [18]:
otchislennie_id_data = dict(iss_bach[~iss_bach['PRICHINA_OTCHISLENIYA'].isin(['болезнь',\
                                                          'нарушение правил прож. в общежитии',\
                                                            'нарушение правил внутр. распорядка',\
                                                                'окончание обуч. по сет. прог. с вузом РФ',\
                                                                    'завершен. программы обуч. академ. обмена'\
                                                                        'призыв в ряды РА',\
                                                                            'перевод в др. ВУЗ', 'окончание обучения'])].dropna(subset='PRICHINA_OTCHISLENIYA')[['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())

akademshiki_id_data = dict(iss_bach[iss_bach['VID_OTPUSKA'].isin(['академический (по семейным обстоятельствам)', 'академический (по болезни)', 'академический (по производственной необходимости)',\
                                                   'академический отпуск (по иным обстоятельствам)'])][['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())


zakonchili_obuchenie_id_data = dict(iss_bach[(iss_bach['PRICHINA_OTCHISLENIYA'] == 'окончание обучения') & (iss_bach['OSNOVANIE_VVODA'] == 'Присвоить')  ][['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())

Удаляем из окончивших обучение тех, кто закончил после академа или отчисления

In [19]:
zakonchili_obuchenie_id_data = {key: value for key, value in zakonchili_obuchenie_id_data.items() if key not in akademshiki_id_data and key not in otchislennie_id_data}

Находим id и date всех бакалавров, ставших магистрами

In [20]:
magistri_id_date = dict(iss_bach[iss_bach['OSNOVANIE_VVODA'] == 'Зачисление в магистратуру'][['LICHNOST_GUID', 'DATA_NACHALA']].groupby('LICHNOST_GUID')['DATA_NACHALA'].min())

Находим всех тех, кто закончил обучение имеет информацию меньше, чем о 8 семестрах обучения

In [21]:
temp_df = srb_fill_nan_ball[(srb_fill_nan_ball['LICHNOST_GUID']).isin(zakonchili_obuchenie_id_data.keys())].groupby('LICHNOST_GUID').nunique()['SEMESTR'] < 8
menshe_8_semstrov = (((temp_df[temp_df == True]).index).to_numpy())


Создаем датафрейм id студентов, которых не будем учитывать

In [22]:
ubrat_id = iss_bach[iss_bach['VID_OTPUSKA'].isin(['в связи с призывом в армию',\
                                        'по уходу за ребенком до 1,5 лет (с продолжением обучения)',\
                                        'по уходу за ребенком до 1,5 лет (без продолжения обучения)',\
                                                   'по беременности и родам',\
                                                    'по уходу за ребенком до 3 лет (без продолжения обучения)',\
                                                        'в связи с загранкомандировкой',\
                                                            'пр.'\
                                                                'по уходу за ребенком до 2 лет (без продолжения обучения)']) | iss_bach['LICHNOST_GUID'].isin(menshe_8_semstrov)]['LICHNOST_GUID']

Собираем id тех, кто ушел в академ, отчислился и выпустился, вместе и убираем из них тех, кого не надо учитывать

In [23]:
akad_otchis_zak_id = pd.DataFrame(np.unique(np.array(list(akademshiki_id_data.keys()) + list(zakonchili_obuchenie_id_data.keys()) + list(otchislennie_id_data.keys()))))
akad_otchis_zak_id = np.array(akad_otchis_zak_id[~akad_otchis_zak_id[0].isin(ubrat_id)]).reshape(-1)

Создаем датафрем записей оценок итоговой группы студентов

In [24]:
srb_tok_nuzhnie_tipi = srb_fill_nan_ball.copy()

srb_tok_nuzhnie_tipi = srb_tok_nuzhnie_tipi[srb_tok_nuzhnie_tipi['LICHNOST_GUID'].isin(akad_otchis_zak_id)]

Находим и удаляем записи об оценках студентов, которые вышли из академа, восстановились, закончили обучение или стали магистрами

In [25]:
srb_tok_nuzhnie_tipi['BALL'] = srb_tok_nuzhnie_tipi.apply(lambda row: -1 if (row['LICHNOST_GUID'] in akademshiki_id_data and akademshiki_id_data[row['LICHNOST_GUID']] < row['DATA_SDACHI']) or \
                                                         (row['LICHNOST_GUID'] in otchislennie_id_data and otchislennie_id_data[row['LICHNOST_GUID']] < row['DATA_SDACHI']) or \
                                                         (row['LICHNOST_GUID'] in zakonchili_obuchenie_id_data and zakonchili_obuchenie_id_data[row['LICHNOST_GUID']] < row['DATA_SDACHI']) or \
                                                           (row['LICHNOST_GUID'] in magistri_id_date and magistri_id_date[row['LICHNOST_GUID']] < row['DATA_SDACHI'])   else row['BALL'], axis=1)

srb_bez_akad_magistr = srb_tok_nuzhnie_tipi.sort_values(by='GOD_NABORA').drop_duplicates(subset=['LICHNOST_GUID', \
                                                   'SEMESTR', \
                                                    'DISCIPLINA', \
                                                    'BALL', \
                                                        'OCENKA', \
                                                            'DATA_SDACHI', \
                                                                'VID_ATT'])\
                                                                    [srb_tok_nuzhnie_tipi['BALL'] > 0]

C:\Users\fluke\AppData\Local\Temp\ipykernel_20452\3009428306.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  srb_bez_akad_magistr = srb_tok_nuzhnie_tipi.sort_values(by='GOD_NABORA').drop_duplicates(subset=['LICHNOST_GUID', \


Мой id **6c37f58a-38ef-4c7e-afd5-031b63300f1c**

In [26]:
srb_vremya_zakritiya_disc = srb_bez_akad_magistr.copy()

In [27]:
def find_semestr_po_date(god_nabora, date):
    days_after_nabor=(date-pd.to_datetime(f'{god_nabora}-07-01')).days
    count_of_semestr=(days_after_nabor//365)*2
    ostatok_of_days= (days_after_nabor%365)
    if ostatok_of_days>=219:
        count_of_semestr+=2
    else:
        count_of_semestr+=1

    return count_of_semestr

In [28]:
srb_vremya_zakritiya_disc['SEMESTR_ZAKRIT_DISC'] = 0

Находим в какой семестр хронологически была закрыта дисциплина (распиши тут про то что чел типа долг закрывает и это пишется в семестр, в котором проходила дисциплина)

In [29]:
srb_vremya_zakritiya_disc['SEMESTR_ZAKRIT_DISC'] = srb_vremya_zakritiya_disc.apply(lambda row: find_semestr_po_date(row['GOD_NABORA'], row['DATA_SDACHI']), axis=1)

Удаляем студентов, у которых информация о годе набора не совпадает с датами проставления оценок

In [30]:
srb_temp = srb_vremya_zakritiya_disc.copy()
# srb_temp = srb_temp[~(srb_temp['LICHNOST_GUID'].isin(srb_temp[srb_temp['SEMESTR_ZAKRIT_DISC'] > 0]['LICHNOST_GUID'].unique()))]
srb_temp = srb_temp[srb_temp['SEMESTR_ZAKRIT_DISC'] > 0]
srb_temp = srb_temp.dropna()

Находим самую популярную дату сдачи предмета у группы и сравниваем ее с датой сдачи студента.
Если дата сдачи совпадает с самой популярной, то в столбце, в котором находится номер семестра хронологически, в котором была закрыта дисциплина, указываем семестр, в котором проходила дисциплина (это для летних практик и других тем)

In [31]:
most_common_dates = srb_temp.groupby(['GRUPPA_GUID', 'DISCIPLINA', 'SEMESTR'])['DATA_SDACHI'].agg(lambda x: x.mode()[0])
srb_temp['SEMESTR_ZAKRIT_DISC'] = srb_temp.apply(lambda row: row['SEMESTR'] if row['DATA_SDACHI'] == most_common_dates[(row['GRUPPA_GUID'], row['DISCIPLINA'], row['SEMESTR'])] else row['SEMESTR_ZAKRIT_DISC'], axis=1)


наша группа  **9967def9-093f-4519-879f-63ca003a301f**

Находим строки, в которых семестр, в котором проходила дисциплина, не совпадает с семестром, в котором эта дисциплина была закрыта.
Добавляем записи об оценках неудовлетворительно, для исправленных в последующем дисциплин. Добавляем столбец, говорящий о том, что дисциплина была исправленна позднее.

In [32]:
srb_temp['ISPRAV'] = 0
srb_temp.loc[srb_temp['SEMESTR']!=srb_temp['SEMESTR_ZAKRIT_DISC'], 'ISPRAV'] = 1
neudi = srb_temp[srb_temp['SEMESTR']!=srb_temp['SEMESTR_ZAKRIT_DISC']]
def make_neud(mean_std_min_max_by_simvol):
    mean, std, min, max = mean_std_min_max_by_simvol[1]['F']

    return round(np.clip(np.random.normal(mean, std), min, max))

neudi['SEMESTR_ZAKRIT_DISC'] = neudi['SEMESTR']
neudi['ISPRAV'] = 0
neudi['BALL'] = [make_neud(mean_std_min_max_by_simvol) for _ in range(neudi.shape[0])]

srb_s_neudami = pd.concat([srb_temp, neudi], axis=0)


C:\Users\fluke\AppData\Local\Temp\ipykernel_20452\1640174403.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neudi['SEMESTR_ZAKRIT_DISC'] = neudi['SEMESTR']
C:\Users\fluke\AppData\Local\Temp\ipykernel_20452\1640174403.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neudi['ISPRAV'] = 0
C:\Users\fluke\AppData\Local\Temp\ipykernel_20452\1640174403.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



Добавляем контекст баллов в группе по дисциплинам.

In [33]:
def add_ball_group_context(data, columns_group, column_context):
    context_data = data.copy()
    context_ways = ['min','max','mean','median']
    for context_trainsform in context_ways:
        context_value = context_data.groupby(columns_group)[column_context].transform(context_trainsform)
        context_data['ball_'+context_trainsform] = context_value
        context_data['ball_'+context_trainsform+'_sub'] = context_data[column_context] - context_value

    max_value_in_group = context_data.groupby(columns_group)[column_context].transform('max')
    context_data['percent_better_than_rest'] = ((max_value_in_group - context_data[column_context]) / max_value_in_group) * 100

    return context_data

srb_context_ball = add_ball_group_context(srb_s_neudami, ['GRUPPA_GUID', 'DISCIPLINA', 'SEMESTR'], 'BALL')

In [34]:
assert len(srb_context_ball[~(srb_context_ball['LICHNOST_GUID'].isin(akademshiki_id_data.keys())) & \
                     ~(srb_context_ball['LICHNOST_GUID'].isin(otchislennie_id_data.keys())) & \
                        ~(srb_context_ball['LICHNOST_GUID'].isin(zakonchili_obuchenie_id_data.keys()))]) == 0

Кодируем столбец обозначающий вид аттестации дисциплины.

In [35]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

vid_att_encoded = encoder.fit_transform(np.array(srb_context_ball['VID_ATT']).reshape(-1, 1))

vid_att_encoded = pd.DataFrame(vid_att_encoded, columns=encoder.get_feature_names_out(['VID_ATT']))

srb_encoded = pd.concat([srb_context_ball.copy().reset_index(drop=True), vid_att_encoded], axis=1)

In [36]:
prp = pd.read_csv(os.path.join(dataPath,'propuski.csv'))
disc = pd.read_csv(os.path.join(dataPath,'discipliny.csv'))

Тут я добавляю нормальные названия дисциплин к айдишникам их. можешь не писать это

In [37]:
prp_norm_disc_names = pd.merge(prp, disc, left_on='DISCIPLINA_ID', right_on='ID').drop(['ID', 'ZAGOLOVOK', 'DATA', 'NOMER_ZANYATIYA'], axis=1)

тут я добавляю айдишник для дисциплин xd

In [38]:
def add_disc_id(srb, disc):
  disc_dict = dict(disc.groupby('FULL_NAME').min()['ID'])
  srb['DISCIPLINA_ID'] = srb.apply(lambda row: disc_dict[row['DISCIPLINA']] if row['DISCIPLINA'] in disc_dict else 0, axis=1)
  return srb

srb_add_disc_id = add_disc_id(srb_encoded.copy(), disc)


Добавляем пропуски, пропуски по виду занятий, и контекст пропусков к 'min', 'max', 'mean', 'median' (на русском распиши действия)

In [39]:
def add_prp_context(prp, srb):
    dict_groupped_prp_data_by_student =  dict(prp.groupby(['LICHNOST_GUID', 'NOMER_SEMESTRA', 'FULL_NAME']).count()['PROPUSK'])

    dict_groupped_prp_data_by_student_by_vid_zan =  dict(prp.groupby(['LICHNOST_GUID', 'NOMER_SEMESTRA', 'FULL_NAME', 'VID_ZAN_ID']).count()['PROPUSK'])

    groupped_prp_data_by_group = prp.groupby(['GRUPPA_GUID', 'LICHNOST_GUID', 'NOMER_SEMESTRA', 'FULL_NAME']).count()\
                                                                                            .groupby(['GRUPPA_GUID', 'NOMER_SEMESTRA', 'FULL_NAME'])

    actions = ['min', 'max', 'mean', 'median']

    dict_actions = {action: dict(groupped_prp_data_by_group.apply(action)['PROPUSK']) for action in actions}

    srb['prp'] = srb.apply(lambda row: dict_groupped_prp_data_by_student[row['LICHNOST_GUID'], row['SEMESTR'], row['DISCIPLINA']] \
                                       if (row['LICHNOST_GUID'], row['SEMESTR'], row['DISCIPLINA']) in dict_groupped_prp_data_by_student \
                                        else 0, axis=1)
    for vid_zan in range(1, 5+1):
      srb['prp_vid_'+str(vid_zan)] = srb.apply(lambda row: dict_groupped_prp_data_by_student_by_vid_zan[row['LICHNOST_GUID'], row['SEMESTR'], row['DISCIPLINA'], vid_zan] \
                                       if (row['LICHNOST_GUID'], row['SEMESTR'], row['DISCIPLINA'], vid_zan) in dict_groupped_prp_data_by_student_by_vid_zan \
                                        else 0, axis=1)

    for action in actions:
        srb[action+'_prp'] = srb.apply(lambda row: dict_actions[action][row['GRUPPA_GUID'], row['SEMESTR'], row['DISCIPLINA']] \
                                       if (row['GRUPPA_GUID'], row['SEMESTR'], row['DISCIPLINA']) in dict_actions[action]\
                                        else 0, axis=1)
        srb['sub_'+action+'_prp'] = srb.apply(lambda row: dict_groupped_prp_data_by_student[row['LICHNOST_GUID'], row['SEMESTR'], row['DISCIPLINA']] - dict_actions[action][row['GRUPPA_GUID'], row['SEMESTR'], row['DISCIPLINA']] \
                                       if (row['GRUPPA_GUID'], row['SEMESTR'], row['DISCIPLINA']) in dict_actions[action] and \
                                          (row['LICHNOST_GUID'], row['SEMESTR'], row['DISCIPLINA']) in dict_groupped_prp_data_by_student \
                                        else 0, axis=1)

    return srb

srb_prp_context = add_prp_context(prp_norm_disc_names, srb_add_disc_id)



In [40]:
srb_prp_context.columns

Index(['GRUPPA_GUID', 'LICHNOST_GUID', 'SEMESTR', 'GOD_NABORA', 'DISCIPLINA',
       'BALL', 'OCENKA', 'SIMVOL', 'DATA_SDACHI', 'VID_ATT',
       'SEMESTR_ZAKRIT_DISC', 'ISPRAV', 'ball_min', 'ball_min_sub', 'ball_max',
       'ball_max_sub', 'ball_mean', 'ball_mean_sub', 'ball_median',
       'ball_median_sub', 'percent_better_than_rest', 'VID_ATT_Зачет',
       'VID_ATT_Курсовая работа', 'VID_ATT_Курсовой проект', 'VID_ATT_Экзамен',
       'DISCIPLINA_ID', 'prp', 'prp_vid_1', 'prp_vid_2', 'prp_vid_3',
       'prp_vid_4', 'prp_vid_5', 'min_prp', 'sub_min_prp', 'max_prp',
       'sub_max_prp', 'mean_prp', 'sub_mean_prp', 'median_prp',
       'sub_median_prp'],
      dtype='object')

In [41]:
srb_prp_context = srb_prp_context[['GRUPPA_GUID', 'LICHNOST_GUID', 'SEMESTR', 'SEMESTR_ZAKRIT_DISC', 'GOD_NABORA', 'DISCIPLINA', 'DISCIPLINA_ID',
       'BALL', 'OCENKA', 'SIMVOL', 'DATA_SDACHI', 'VID_ATT',
        'VID_ATT_Зачет', 'VID_ATT_Курсовая работа', 'VID_ATT_Курсовой проект',
       'VID_ATT_Экзамен', 'ISPRAV', 'ball_min', 'ball_min_sub', 'ball_max',
       'ball_max_sub', 'ball_mean', 'ball_mean_sub', 'ball_median',
       'ball_median_sub', 'percent_better_than_rest',
         'prp', 'prp_vid_1', 'prp_vid_2', 'prp_vid_3',
       'prp_vid_4', 'prp_vid_5', 'min_prp', 'sub_min_prp', 'max_prp',
       'sub_max_prp', 'mean_prp', 'sub_mean_prp', 'median_prp',
       'sub_median_prp']]

In [42]:
disc_chasy = pd.read_csv(os.path.join(dataPath, 'disciplina_chasy.csv'))

Добавляю часы отведенные на дисциплину для группы студентов

In [43]:
srb_chasy = pd.merge(srb_prp_context, disc_chasy, left_on=['GRUPPA_GUID', 'DISCIPLINA_ID'], right_on=['GRUPPA_GUID', 'DISCIPLINA_ID'])


,GRUPPA_GUID,LICHNOST_GUID,SEMESTR,SEMESTR_ZAKRIT_DISC,GOD_NABORA,DISCIPLINA,DISCIPLINA_ID,BALL,OCENKA,SIMVOL,...,max_prp,sub_max_prp,mean_prp,sub_mean_prp,median_prp,sub_median_prp,NOMER_DISC,AUD,SAM,VSEGO
0,f35ebfb8-5d8a-41da-872d-1693b50463c4,7344ee4f-8c42-44e5-a67f-af326930bd49,6,6.0,2010,Военная подготовка,27,95.0,Отлично,A,...,0,0,0.000000,0.000000,0.0,0.0,1,0.0,0.0,0.0
1,f35ebfb8-5d8a-41da-872d-1693b50463c4,7344ee4f-8c42-44e5-a67f-af326930bd49,8,8.0,2010,Военная подготовка,27,100.0,Отлично,A+,...,0,0,0.000000,0.000000,0.0,0.0,1,0.0,0.0,0.0
2,f35ebfb8-5d8a-41da-872d-1693b50463c4,7344ee4f-8c42-44e5-a67f-af326930bd49,7,7.0,2010,Военная подготовка,27,85.0,Зачет,D,...,0,0,0.000000,0.000000,0.0,0.0,1,0.0,0.0,0.0
3,f35ebfb8-5d8a-41da-872d-1693b50463c4,763bf113-4134-4c82-a2b8-24b1ccdbc452,7,7.0,2010,Военная подготовка,27,85.0,Зачет,D,...,0,0,0.000000,0.000000,0.0,0.0,1,0.0,0.0,0.0
4,f35ebfb8-5d8a-41da-872d-1693b50463c4,763bf113-4134-4c82-a2b8-24b1ccdbc452,8,8.0,2010,Военная подготовка,27,100.0,Отлично,A+,...,0,0,0.000000,0.000000,0.0,0.0,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793728,dcf48f7b-94af-4ba5-b22a-7f18451d12ec,aacd0d52-b10d-4ac2-a880-30b84ab9c922,1,1.0,2023,Иностранный язык (английский),9,19.0,Незачет,F,...,32,-17,11.038462,3.961538,6.5,8.5,6,128.0,88.0,216.0
793729,dcf48f7b-94af-4ba5-b22a-7f18451d12ec,aacd0d52-b10d-4ac2-a880-30b84ab9c922,1,1.0,2023,Начертательная геометрия и инженерная графика 1.1,9760,21.0,Неудовлетворительно,F,...,27,-17,8.586207,1.413793,4.0,6.0,11,48.0,96.0,144.0
793730,dcf48f7b-94af-4ba5-b22a-7f18451d12ec,aacd0d52-b10d-4ac2-a880-30b84ab9c922,1,1.0,2023,Основы российской государственности,54426,56.0,Удовлетворительно,E,...,11,-6,4.928571,0.071429,4.0,1.0,2,56.0,16.0,72.0
793731,7a6babdb-7e67-4884-99ff-878a549bf61a,907b8e83-101e-47d2-a3f5-c3172cf339fc,1,1.0,2023,Математическая логика и теория алгоритмов,2229,7.0,Неудовлетворительно,F,...,23,-3,5.724138,14.275862,5.0,15.0,3,56.0,124.0,180.0


In [44]:
srb_chasy.columns

Index(['GRUPPA_GUID', 'LICHNOST_GUID', 'SEMESTR', 'SEMESTR_ZAKRIT_DISC',
       'GOD_NABORA', 'DISCIPLINA', 'DISCIPLINA_ID', 'BALL', 'OCENKA', 'SIMVOL',
       'DATA_SDACHI', 'VID_ATT', 'VID_ATT_Зачет', 'VID_ATT_Курсовая работа',
       'VID_ATT_Курсовой проект', 'VID_ATT_Экзамен', 'ISPRAV', 'ball_min',
       'ball_min_sub', 'ball_max', 'ball_max_sub', 'ball_mean',
       'ball_mean_sub', 'ball_median', 'ball_median_sub',
       'percent_better_than_rest', 'prp', 'prp_vid_1', 'prp_vid_2',
       'prp_vid_3', 'prp_vid_4', 'prp_vid_5', 'min_prp', 'sub_min_prp',
       'max_prp', 'sub_max_prp', 'mean_prp', 'sub_mean_prp', 'median_prp',
       'sub_median_prp', 'NOMER_DISC', 'AUD', 'SAM', 'VSEGO'],
      dtype='object')

In [45]:
srb_chasy = srb_chasy.rename(columns={'VSEGO': 'DISC_CHASY', 'AUD': 'AUD_DISC_CHASY', 'SAM': 'SAM_DISC_CHASY',})

Добавляю соотношение пропусков к часам отведенным на занятия в аудитории

In [46]:
def add_proc_prp_context(srb):

    srb['prp_to_aud_chasy_ratio'] = srb.apply(lambda row: row['prp'] / row['AUD_DISC_CHASY'] if (row['AUD_DISC_CHASY'] != 0) else 0, axis=1)

    for vid_zan in range(1, 5+1):
      srb['prp_vid_'+str(vid_zan)+'_to_aud_chasy_ratio'] = srb.apply(lambda row: row['prp_vid_'+str(vid_zan)] / row['AUD_DISC_CHASY'] if (row['AUD_DISC_CHASY'] != 0) else 0, axis=1)

    return srb

srb_prp_ratio = add_proc_prp_context(srb_chasy.copy())

Тут я добавлю еще тип финансирования

In [47]:
itfs = pd.read_csv(os.path.join(dataPath, 'ist_tip_fin_studenta.csv'))
itfs['DATA_NACHALA'] = pd.to_datetime(itfs['DATA_NACHALA'], errors='coerce')
itfs

,GRUPPA_GUID,LICHNOST_GUID,DATA_NACHALA,TIP_FINANSIROVANIYA_ID,TIP_FINANSIROVANIYA,TIP_OSNOVANIYA_VVODA_ID,PRICHINA_VVODA_ZAPISI,TIP_PARAGRAFA_PRIKAZA_ID,OSNOVANIE_VVODA
0,b5cdf999-9ad9-4d7f-82d2-14a2a8240fc3,fa5af205-36d1-4562-a485-68ac1ff27ac2,2002-09-01,3,Бюджетная,2,приказ,70.0,Зачисление на 1 курс
1,bf266059-5d14-4f01-be4f-890fec36424f,fa5af205-36d1-4562-a485-68ac1ff27ac2,2005-09-15,2,Договорная,0,пр,70.0,Зачисление на 1 курс
2,232c90f1-6fa6-4552-b473-7b42cd1ee8a8,08ab8dd1-7589-44f0-ad2e-83fedf2019e9,2008-12-01,2,Договорная,0,пр,70.0,Зачисление на 1 курс
3,a8c2daa3-4e86-4977-90a2-a4929c89d306,08ab8dd1-7589-44f0-ad2e-83fedf2019e9,2002-09-01,3,Бюджетная,2,приказ,70.0,Зачисление на 1 курс
4,20834d04-deaf-4f52-b172-0572571fe6e0,08ab8dd1-7589-44f0-ad2e-83fedf2019e9,2002-10-01,2,Договорная,2,приказ,70.0,Зачисление на 1 курс
...,...,...,...,...,...,...,...,...,...
165359,548513a4-a7a4-408a-bd5a-489bd99e5efc,28eae6c3-4c66-4613-9265-b55561981b65,2023-09-01,3,Бюджетная,2,приказ,70.0,Зачисление на 1 курс
165360,3ac7b23f-26f1-4f75-b3a0-268b353fd602,10438414-e973-4764-bc7d-7f40b45868cb,2023-09-01,3,Бюджетная,2,приказ,70.0,Зачисление на 1 курс
165361,548513a4-a7a4-408a-bd5a-489bd99e5efc,3b98ce5a-b377-4310-81f0-d4ff12f0d792,2023-09-01,3,Бюджетная,2,приказ,70.0,Зачисление на 1 курс
165362,0ed68b57-e451-44e7-89c1-32174125e6ff,eadf3acc-05db-47bd-9bcd-9d1c82edaec3,2023-09-25,3,Бюджетная,0,пр,83.0,Зачисление в число слушателей


In [48]:
itfs[['LICHNOST_GUID', 'TIP_FINANSIROVANIYA']].shape, itfs[['LICHNOST_GUID', 'TIP_FINANSIROVANIYA']].drop_duplicates().shape

((165364, 2), (133369, 2))

In [49]:
itfs_god_nabora = pd.merge(itfs, srb_prp_ratio[['LICHNOST_GUID', 'GOD_NABORA']].drop_duplicates(), left_on=['LICHNOST_GUID'], right_on=['LICHNOST_GUID'])

In [50]:
itfs_god_nabora['OSNOVANIE_VVODA'].value_counts()

OSNOVANIE_VVODA
Зачисление на 1 курс                        18429
Зачисление в магистратуру                    7570
Восстановить                                  862
Зачисление в число слушателей                 713
Перевести                                     498
Зачисление в порядке перевода из др.ВУЗА      151
Зачисление студента, отчисл. из др.ВУЗА        39
Зачисление на 2-е высшее образ.                29
Восстановить из др. ВУЗа                        2
Зачисление из др. ВУЗа по совм. ОП              1
Name: count, dtype: int64

In [51]:
itfs_god_nabora = itfs_god_nabora[~itfs_god_nabora['OSNOVANIE_VVODA'].isin(['Зачисление в магистратуру', 'Восстановить', 'Зачисление студента, отчисл. из др.ВУЗА', \
                                                                            'Зачисление в число слушателей', 'Перевести'])]

In [52]:
itfs_god_nabora[['LICHNOST_GUID']].drop_duplicates().shape, itfs_god_nabora[['LICHNOST_GUID', 'TIP_FINANSIROVANIYA']].drop_duplicates().shape

((14332, 1), (14709, 2))

In [53]:
slushatel_id = itfs[itfs['OSNOVANIE_VVODA'].isin(['Зачисление в число слушателей'])][['LICHNOST_GUID']].drop_duplicates()

In [54]:
tip_finans = itfs_god_nabora[['LICHNOST_GUID', 'TIP_FINANSIROVANIYA']].drop_duplicates()

In [55]:
lichnost = pd.read_csv(os.path.join(dataPath, 'lichnost.csv'))

In [56]:
lichnost_tip_finans = pd.merge(tip_finans, lichnost, left_on='LICHNOST_GUID', right_on='GUID').drop(['GUID'], axis=1)

In [57]:
lichnost_god_nabora = pd.DataFrame(srb_prp_ratio[['LICHNOST_GUID', 'GOD_NABORA']].groupby('LICHNOST_GUID')['GOD_NABORA'].min()).reset_index()

In [58]:
lichnost_tip_finans_god_nabora = pd.merge(lichnost_tip_finans, lichnost_god_nabora, on='LICHNOST_GUID')

In [59]:
lichnost_tip_finans_god_nabora['AGE_POSTUPLENIYA'] = lichnost_tip_finans_god_nabora['GOD_NABORA'] - lichnost_tip_finans_god_nabora['GOD_ROZHDENIYA']

In [60]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

tip_finans_encoded = encoder.fit_transform(np.array(lichnost_tip_finans_god_nabora['TIP_FINANSIROVANIYA']).reshape(-1, 1))

tip_finans_encoded = pd.DataFrame(tip_finans_encoded, columns=encoder.get_feature_names_out(['TIP_FINANSIROVANIYA']))

lichnost_tip_finans_god_nabora_encoded = pd.concat([lichnost_tip_finans_god_nabora.copy().reset_index(drop=True), tip_finans_encoded], axis=1).drop(['TIP_FINANSIROVANIYA', 'GOD_ROZHDENIYA', 'GOD_NABORA'], axis=1)

In [61]:
lichnost_tip_finans_god_nabora_encoded.loc[lichnost_tip_finans_god_nabora_encoded['POL_ID'] == 2, 'POL_ID'] = 0

In [62]:
lichnost_tip_finans_god_nabora_encoded['SLUSHATEL'] = 0

lichnost_tip_finans_god_nabora_encoded.loc[lichnost_tip_finans_god_nabora_encoded['LICHNOST_GUID'].isin(slushatel_id), 'SLUSHATEL'] = 1

In [63]:
lichnost_tip_finans_god_nabora_encoded

,LICHNOST_GUID,POL_ID,AGE_POSTUPLENIYA,TIP_FINANSIROVANIYA_Договорная,TIP_FINANSIROVANIYA_Целевой приём,SLUSHATEL
0,36233cc2-56a6-4750-a7a5-1d436a5a2cf3,0,18.0,0.0,0.0,0
1,fea27ae2-996a-4533-a1af-6a95ac7b3cfc,0,18.0,0.0,0.0,0
2,593dedc2-5178-4042-8c65-e9fffde065e9,1,18.0,0.0,1.0,0
3,225183c5-ea65-40bd-8380-80fcfc32219a,1,18.0,0.0,0.0,0
4,07f8bf68-8170-4a82-b5cb-2447a141feb7,0,18.0,0.0,0.0,0
...,...,...,...,...,...,...
14704,0f8ba0af-554f-416d-8db7-26f0ce401f97,1,18.0,0.0,0.0,0
14705,18d29c9c-73bd-422c-b721-d21cfd0a9a39,1,19.0,0.0,0.0,0
14706,64eee842-3c1c-4fe1-95ad-2d17bd5d9b61,1,18.0,0.0,0.0,0
14707,24ed7239-c606-4f4b-bc9e-2faf1f53f2b3,1,18.0,0.0,0.0,0


In [64]:
srb_lichnost_info = pd.merge(lichnost_tip_finans_god_nabora_encoded, srb_prp_ratio, on='LICHNOST_GUID')

In [65]:
srb_final = srb_prp_ratio.drop(['GRUPPA_GUID', 'SEMESTR', 'GOD_NABORA', 'DISCIPLINA', 'DISCIPLINA_ID', 'OCENKA', 'SIMVOL', 'DATA_SDACHI', 'VID_ATT', 'NOMER_DISC', 'AUD_DISC_CHASY', 'SAM_DISC_CHASY'], axis=1)

In [66]:
srb_final = srb_final[~(srb_final['LICHNOST_GUID'].isin(srb_final[srb_final['SEMESTR_ZAKRIT_DISC'] > 8]['LICHNOST_GUID']))]

In [67]:
srb_final.groupby('LICHNOST_GUID')['SEMESTR_ZAKRIT_DISC'].max().value_counts().sort_index()

SEMESTR_ZAKRIT_DISC
1.0    1133
2.0    1693
3.0    1060
4.0     746
5.0     531
6.0     389
7.0     242
8.0    8509
Name: count, dtype: int64

In [68]:
srb_final[srb_final['LICHNOST_GUID'].isin(otchislennie_id_data.keys() | akademshiki_id_data.keys())].groupby('LICHNOST_GUID')['SEMESTR_ZAKRIT_DISC'].max().value_counts().sort_index()

SEMESTR_ZAKRIT_DISC
1.0    1133
2.0    1693
3.0    1060
4.0     746
5.0     531
6.0     385
7.0     242
8.0     510
Name: count, dtype: int64

In [69]:
srb_final[srb_final['LICHNOST_GUID'].isin(zakonchili_obuchenie_id_data.keys())].groupby('LICHNOST_GUID')['SEMESTR_ZAKRIT_DISC'].max().value_counts().sort_index()

SEMESTR_ZAKRIT_DISC
6.0       4
8.0    7999
Name: count, dtype: int64

In [70]:
iss_bach[iss_bach['LICHNOST_GUID'] == 'fff9de01-72a2-45d7-9d42-8ae2474e9a69']

,GRUPPA_GUID,LICHNOST_GUID,DATA_NACHALA,STATUS_STUDENTA_ID,STATUS,TIP_OSNOVANIYA_VVODA_ID,PRICHINA_VVODA_ZAPISI,PRICHINA_OTCHISLENIYA_ID,PRICHINA_OTCHISLENIYA,VID_OTPUSKA_ID,VID_OTPUSKA,TIP_PARAGRAFA_PRIKAZA_ID,OSNOVANIE_VVODA,KATEGORIYA_OBUCHAEMOGO_ID,KATEGORIYA_OBUCHAEMOGO
245011,2ee62aa9-e2e0-4728-a96b-cd40b02e0105,fff9de01-72a2-45d7-9d42-8ae2474e9a69,2020-08-31,15,отчисленный,0,пр,13.0,окончание обучения,NaN,NaN,18.0,Присвоить,NaN,NaN
245012,2ee62aa9-e2e0-4728-a96b-cd40b02e0105,fff9de01-72a2-45d7-9d42-8ae2474e9a69,2020-07-06,16,в отпуске,0,пр,NaN,NaN,11.0,каникулы,18.0,Присвоить,NaN,NaN
245013,2ee62aa9-e2e0-4728-a96b-cd40b02e0105,fff9de01-72a2-45d7-9d42-8ae2474e9a69,2016-08-29,10,учащийся,0,пр,NaN,NaN,NaN,NaN,70.0,Зачисление на 1 курс,NaN,NaN


In [74]:
srb_final[srb_final['LICHNOST_GUID'].isin(otchislennie_id_data.keys() | akademshiki_id_data.keys())].groupby('LICHNOST_GUID')['SEMESTR_ZAKRIT_DISC'].max()

LICHNOST_GUID
000f238b-ca7e-4ceb-83fa-80360cb049f9    3.0
001c6af2-42be-44db-89d1-46d240bc4518    1.0
0022fae9-81a2-4a27-8064-33c0b098577f    1.0
00258832-84ad-42bc-865d-cb84240fa332    1.0
002774d1-ad0e-4741-9b77-b0f28f27c804    3.0
                                       ... 
ffc88289-af7a-4419-83c8-072b58a38a40    2.0
ffca1d13-bad7-44c9-9a90-e565f95074de    2.0
ffd020a5-e83c-4238-8581-ad87e2d09bbf    2.0
ffd23ff2-310e-4d82-b1a6-33eb9aea89d4    2.0
ffd46eaa-99ff-4610-8c83-c4c09373f621    1.0
Name: SEMESTR_ZAKRIT_DISC, Length: 6300, dtype: float64

In [75]:
srb_final.to_csv(os.path.join(preprocessedDataPath, 'final.csv'), index=False)

srb_final

,LICHNOST_GUID,SEMESTR_ZAKRIT_DISC,BALL,VID_ATT_Зачет,VID_ATT_Курсовая работа,VID_ATT_Курсовой проект,VID_ATT_Экзамен,ISPRAV,ball_min,ball_min_sub,...,sub_mean_prp,median_prp,sub_median_prp,DISC_CHASY,prp_to_aud_chasy_ratio,prp_vid_1_to_aud_chasy_ratio,prp_vid_2_to_aud_chasy_ratio,prp_vid_3_to_aud_chasy_ratio,prp_vid_4_to_aud_chasy_ratio,prp_vid_5_to_aud_chasy_ratio
0,7344ee4f-8c42-44e5-a67f-af326930bd49,6.0,95.0,0.0,0.0,0.0,1.0,0,80.0,15.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,7344ee4f-8c42-44e5-a67f-af326930bd49,8.0,100.0,0.0,0.0,0.0,1.0,0,80.0,20.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
2,7344ee4f-8c42-44e5-a67f-af326930bd49,7.0,85.0,1.0,0.0,0.0,0.0,0,85.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
3,763bf113-4134-4c82-a2b8-24b1ccdbc452,7.0,85.0,1.0,0.0,0.0,0.0,0,85.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
4,763bf113-4134-4c82-a2b8-24b1ccdbc452,8.0,100.0,0.0,0.0,0.0,1.0,0,80.0,20.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793728,aacd0d52-b10d-4ac2-a880-30b84ab9c922,1.0,19.0,1.0,0.0,0.0,0.0,0,19.0,0.0,...,3.961538,6.5,8.5,216.0,0.117188,0.000000,0.0,0.117188,0.0,0.0
793729,aacd0d52-b10d-4ac2-a880-30b84ab9c922,1.0,21.0,0.0,0.0,0.0,1.0,0,21.0,0.0,...,1.413793,4.0,6.0,144.0,0.208333,0.020833,0.0,0.187500,0.0,0.0
793730,aacd0d52-b10d-4ac2-a880-30b84ab9c922,1.0,56.0,0.0,0.0,0.0,1.0,0,56.0,0.0,...,0.071429,4.0,1.0,72.0,0.089286,0.089286,0.0,0.000000,0.0,0.0
793731,907b8e83-101e-47d2-a3f5-c3172cf339fc,1.0,7.0,0.0,0.0,0.0,1.0,0,7.0,0.0,...,14.275862,5.0,15.0,180.0,0.357143,0.160714,0.0,0.196429,0.0,0.0


In [76]:
pd.DataFrame(zakonchili_obuchenie_id_data.keys()).to_csv(os.path.join(preprocessedDataPath,'zakonchili_obuchenie_id.csv'), index=False)
pd.DataFrame(akademshiki_id_data.keys()).to_csv(os.path.join(preprocessedDataPath,'akademshiki_id.csv'), index=False)
pd.DataFrame(otchislennie_id_data.keys()).to_csv(os.path.join(preprocessedDataPath,'otchislennie_id.csv'), index=False)

pd.DataFrame(lichnost_tip_finans_god_nabora_encoded).to_csv(os.path.join(preprocessedDataPath,'lichnost.csv'), index=False)

